In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="L92yodulFT4LlHfg1I5l")
project = rf.workspace("aiclimbing").project("hold-detector")
dataset = project.version(12).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Hold-Detector-12 in yolov8:: 100%|█████████████████| 84/84 [00:00<00:00, 2476.39it/s]


In [3]:
from ultralytics import YOLO 
import os
# Training.
model = YOLO("yolov8n.pt")
try:
    os.rename("datasets/data.yaml", "data.yaml")
except:
    pass
results = model.train(
    data='data.yaml',
    imgsz=1280,
    epochs=50,
    batch=8,
    name='yolov8n_v8_50e',
)

New https://pypi.org/project/ultralytics/8.0.220 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.8.18 torch-2.1.1 CPU (Intel Core(TM) i7-10875H 2.30GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=8, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v8_50e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnosti


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       8/50         0G      1.541      1.765      1.337         22       1280: 100%|██████████| 4/4 [00:31<00:00,  7.76
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<0
                   all          7        139     0.0229      0.345      0.276       0.18

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       9/50         0G      1.291      2.103      1.314          3       1280: 100%|██████████| 4/4 [00:33<00:00,  8.45
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<0
                   all          7        139     0.0243      0.367      0.302      0.193

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      10/50         0G      1.621      1.521      1.264         42       1280: 100%|██████████| 4/4 [00:3

      26/50         0G      1.765      2.118      1.432          3       1280: 100%|██████████| 4/4 [00:32<00:00,  8.00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?WARNING  NMS time limit 0.850s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<0
                   all          7        139      0.801      0.435      0.577      0.296

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      27/50         0G      1.353      1.315      1.176         30       1280: 100%|██████████| 4/4 [00:33<00:00,  8.27
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?WARNING  NMS time limit 0.850s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<0
                   all          7 

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<0
                   all          7        139      0.773      0.561      0.671      0.362
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      41/50         0G      1.406      2.764      1.686          1       1280: 100%|██████████| 4/4 [00:32<00:00,  8.08
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?WARNING  NMS time limit 0.850s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<0
                   all          7        139      0.792      0.583      0.697      0.372

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      42/50         0G      1.408      1.596      1.302          3       1280: 100%|██████████| 4/4 [00:32<00:00,  8.20

In [7]:
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/yolov8n_v8_50e19/weights/best.pt')

# Run inference on 'bus.jpg'
results = model('hold.jpg')  # results list
print(results.boxes)

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image


image 1/1 C:\Users\liaml\AI-Climbing\hold.jpg: 1280x960 7 holds, 196.5ms
Speed: 12.0ms preprocess, 196.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)


AttributeError: 'list' object has no attribute 'boxes'